In [1]:

import math 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.signal import decimate, resample_poly, resample
from scipy.interpolate import PchipInterpolator

from ahrs.filters.madgwick import Madgwick #ahrs library requires git clone, do not pip install


In [ ]:
def imu_processing(xl1, xl1_interval_ids, mg, gy, interval_odr, gyr_offset, og_odr, sub_mg):
    """
    Main function, group data by intervals, build window of adjacent intervals for time alignment, this fixes large values on ends
    """

    sensor_odr = {"xl":1000, 'gy':1000, 'mg':20}
    sensor_dt = {'xl':.001, 'gy':.001, 'mg':.05}

    cols =['interval','timestamp','x', 'y', 'z']

    # xl1_df = pd.DataFrame(xl1, columns=cols)
    # xl1_group = xl1_df.groupby('interval')

    
    mg_df = pd.DataFrame(mg, columns=cols)
    gy_df = pd.DataFrame(gy, columns=cols)

    n_intervals = len(np.unique(xl1_interval_ids))
    interval_ids = np.repeat(np.arange(998), 1000)

    xl1_mod = np.column_stack([xl1, interval_ids])
    xl1_df = pd.DataFrame(xl1_mod, columns=['x','y','z','timestamp', 'interval'])
    
    xl1_group = xl1_df.groupby('interval')
    mg_group = mg_df.groupby('interval')
    gy_group = gy_df.groupby('interval')

    xl_global = []
    prev_q = [1,0,0,0]

    xl_w = [] #(3, n, 3) history of 3, n data , xyz 
    gy_w = []
    mg_w = []

    for (name1, xl1_grp), (name2, mg_grp), (name3, gy_grp) in zip(xl1_group, mg_group, gy_group):
        assert name1 == name2 == name3, f"Group names don't match: {name1}, {name2}, {name3}" #groups all must belong to the same interval 

        # gyr_offset = np.array([.58, .42, -.3])
        # xl_ = preprocess_sensor(xl1_grp.drop(columns=['interval']).values)
        mg_ = preprocess_sensor(mg_grp.drop(columns=['interval']).values)
        gy_ = preprocess_sensor(gy_grp.drop(columns=['interval']).values) #remove unique values 
        # xl_p, mg_p, gy_p = preprocess_imu(xl_1[:,1:], mg_[:,1:], gy_[:,1:], gyr_offset, 2, clip=False)
        xl_w.append(xl1_grp.drop(columns=['interval', 'timestamp']).values)
        mg_w.append(mg_[:,1:])
        gy_w.append(gy_[:,1:])

        if len(xl_w) > 3: 
            xl_w.pop(0)
            mg_w.pop(0)
            gy_w.pop(0)
    
        if len(xl_w) != 3:
            continue 
            
        interval_q = []

        if og_odr == True: 
            interpolate = False

            # xl_t = time_align(xl_w, 1000, 1000, 'xl', interpolate)
            xl_t = np.array(xl_w).reshape(-1,3)[interval_odr:2*interval_odr]
            mg_t = time_align(mg_w, 20, 20, 'mg', interpolate)
            gy_t = time_align(gy_w, 1000, 1000, 'gy', interpolate)

            xl_p, mg_p, gy_p = preprocess_imu(xl_t, mg_t, gy_t, gyr_offset, 2, clip=False)
            mg_idx = 0  
            q_i = prev_q

            for i in range(xl_t.shape[0]):

                if i % (xl_t.shape[0]//20) == 0 or i==0:

                    mg_curr = np.expand_dims(mg_p[mg_idx, :], axis=0)
                    xl_curr = np.expand_dims(xl_p[i, :], axis=0) 
                    gy_curr = np.expand_dims(gy_p[i, :], axis=0)
                    
                    if sub_mg == True:
                        model = Madgwick(acc=xl_curr, gyr=gy_curr, mag=mg_curr, dt=1/interval_odr, q0 = q_i)
                    
                    else:
                        model = Madgwick(acc=xl_curr, gyr=gy_curr, dt=1/interval_odr, q0 = q_i)
                    q_i = model.Q[-1]
                    mg_idx += 1
                
                else: 
                    xl_curr = np.expand_dims(xl_p[i, :], axis=0)
                    gy_curr = np.expand_dims(gy_p[i, :], axis=0)
                    model = Madgwick(acc=xl_curr, gyr=gy_curr, dt=1/interval_odr, q0 = q_i)
                    q_i = model.Q[-1]

                interval_q.append(q_i)

            prev_q = q_i
        
        else:
            interpolate = True

            xl_t = time_align(xl_w, 1000, interval_odr, 'xl', interpolate)
            mg_t = time_align(mg_w, 20, interval_odr, 'mg', interpolate)
            gy_t = time_align(gy_w, 1000, interval_odr, 'gy', interpolate)

            model = Madgwick(acc=xl_t, mag=mg_t, gyr=gy_t, frequency = interval_odr, q0 = prev_q)
            prev_q = model.Q[-1]
            interval_q = model.Q

        group_xl_ned = []
        for q,x in zip(np.array(interval_q), xl_t):
            x*=9.8
            mag = np.linalg.norm(x)
            norm_vec = x / mag
            ned_vec_1 = rotate_to_ned(q, norm_vec)
            vec1 = np.round(ned_vec_1,4)

            q = normalize(q)
            ned_vec_2 = rotate_vector(q, norm_vec, active=False)
            vec2 = np.round(ned_vec_2,4)

            assert np.allclose(vec1, vec2) , f"Not equal vec1: {vec1}, vec2: {vec2}"

            ned_vec_1 *= mag
            ned_vec_1[2] -= 9.810665
            # xl_global.append(ned_vec_1)
            group_xl_ned.append(ned_vec_1)
        
        group_ts = [xl1_grp.iloc[xl1_grp.shape[0]//2]['timestamp'] for _ in range(np.array(group_xl_ned).shape[0])]
        group_interval = [xl1_grp.iloc[xl1_grp.shape[0]//2]['interval'] for _ in range(np.array(group_xl_ned).shape[0])]

        group_xl_ned = np.column_stack([group_xl_ned, group_ts, group_interval])
        xl_global.append(group_xl_ned)

        # print(np.array(xl_grp).shape)
        # plt.plot(np.arange(gy_t.shape[0]), gy_t[:,2]-9.8)
        # plt.show()
        # plt.plot(np.array(gy_t)[:,2])
        # plt.show()
        # print(np.array(gy_t).shape[0])
        # print('dfdf')
        # break

    return np.vstack(xl_global)